In [1]:
log_root = r'D:\issp_data\features_total_final_uncompressed'
fraction = 1

model_folder = r'D:\issp_data\model'
# test_size = 0.25
start_month_train, end_month_train = 4, 5
start_month_test, end_month_test = 6, 6

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')

from typing import Tuple, List

import dask.dataframe as dd
import gzip
import numpy as np
import pickle
import time

from distributed.protocol.serialize import register_generic
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tpot import TPOTClassifier
register_generic(TPOTClassifier)

import xgboost as xgb

In [3]:
shift = 0

# max(ts)
# min(ts)
# diff(ts) = max(ts) - min(ts)
# len(ts)
# diff(ts) / len(ts) --> density
# max(dr)
# min(dr)
# mean(dr)
# median(dr)
# var(dr)
selected = [r'ts_max', r'ts_min', r'ts_diff', r'ts_len', r'ts_density',
            r'dr_max', r'dr_min', r'dr_mean', r'dr_median', r'dr_var']

def load_data(st: int, ed: int) -> Tuple['np.array']:
    path = os.path.join(log_root, sorted(os.listdir(log_root))[0])
#     col_nums = dd.read_csv(path, compression='zip').shape[-1]
#     col_nums = dd.read_csv(path).shape[-1] - shift
    col_nums = dd.read_csv(path, usecols=selected).shape[-1] - shift
    
    arr_X, arr_y = np.zeros(col_nums).reshape((-1, col_nums)), np.array([0])
    
    for fl in os.listdir(log_root)[(st - 1) * 3: ed * 3]:
        cata = int(fl[31])
        path = os.path.join(log_root, fl)
        print(path)
#         tmp = dd.read_csv(path, compression='zip')\
#         tmp = dd.read_csv(path)\
        
        tmp = dd.read_csv(path, usecols=selected)\
                .compute()\
                .values[:, shift:]
        print(tmp.shape)
        
        arr_X = np.vstack((arr_X, tmp))
        arr_y = np.hstack((arr_y, np.array([cata] * tmp.shape[0])))
        
    return arr_X[1: , :], arr_y[1: ]

In [4]:
# shift = 2

# def load_data(st: int, ed: int) -> Tuple['np.array']:
#     path = os.path.join(log_root, sorted(os.listdir(log_root))[0])
# #     col_nums = dd.read_csv(path, compression='zip').shape[-1]
#     col_nums = dd.read_csv(path).shape[-1] - shift
    
#     arr_X, arr_y = np.zeros(col_nums).reshape((-1, col_nums)), np.array([0])
    
#     for fl in os.listdir(log_root)[(st - 1) * 3: ed * 3]:
#         cata = int(fl[31])
#         path = os.path.join(log_root, fl)
#         print(path)
# #         tmp = dd.read_csv(path, compression='zip')\
#         tmp = dd.read_csv(path)\
#                 .compute()\
#                 .values[:, shift:]
        
#         arr_X = np.vstack((arr_X, tmp))
#         arr_y = np.hstack((arr_y, np.array([cata] * tmp.shape[0])))
        
#     return arr_X[1: , :], arr_y[1: ]

In [5]:
def feature_importances(model_obj) -> List:
    path = lambda x: os.path.join(x, os.listdir(x)[0])
#     df = dd.read_csv(path(log_root), sep='\t', compression='zip')
    df = dd.read_csv(path(log_root), sep='\t')
#     cols = list(df.compute())[0].split(',')[shift:]
    cols = selected

    features = model_obj.feature_importances_\
                    .tolist()
    result = [(fea, col) for (fea, col) in zip(features, cols)]
    return sorted(result, reverse=True)

In [6]:
# arr_X, arr_y = load_data(1, 12)

In [7]:
# X, y = resample(arr_X, arr_y, n_samples=arr_X.shape[0]//fraction,
#                 replace=False, stratify=arr_y, random_state=None)

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=test_size, random_state=None)

# print(X_train.shape, y_train.shape)
# print(X_test.shape, y_test.shape)

In [8]:
# import train data
arr_X, arr_y = load_data(start_month_train, end_month_train)
X_train, y_train = resample(arr_X, arr_y, n_samples=arr_X.shape[0]//fraction,
                            replace=False, stratify=arr_y, random_state=None)

print('\ntotal:\n', X_train.shape, y_train.shape)

D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201704_104647_0_features.csv
(313307, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201704_104647_1_features.csv
(185061, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201704_104647_2_features.csv
(127174, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201705_104648_0_features.csv
(328907, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201705_104648_1_features.csv
(190919, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201705_104648_2_features.csv
(132772, 10)

total:
 (426, 10) (426,)


In [9]:
# import test data
arr_X, arr_y = load_data(start_month_test, end_month_test)
X_test, y_test = resample(arr_X, arr_y, n_samples=arr_X.shape[0]//fraction,
                            replace=False, stratify=arr_y, random_state=None)

print('\ntotal:\n', X_test.shape, y_test.shape)

D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201706_104649_0_features.csv
(291474, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201706_104649_1_features.csv
(161097, 10)
D:\issp_data\features_total_final_uncompressed\s35809_MobileApp_201706_104649_2_features.csv
(119131, 10)

total:
 (190, 10) (190,)


In [10]:
# config_dict = {
#     'xgboost.XGBClassifier': {
#     },
#     'sklearn.ensemble.ExtraTreesClassifier': {
#     },
#     'sklearn.ensemble.GradientBoostingClassifier': {
#     }
# }

# clf = TPOTClassifier(generations=3,
#                      population_size=10, 
#                      n_jobs=-1,
#                      verbosity=2,
#                      config_dict=config_dict,
#                      random_state=None,
#                      use_dask=True)

# %time clf.fit(X_train, y_train)
# print(clf.fitted_pipeline_.steps[1])

# print(clf.score(X_test, y_test))

In [11]:
# Random forest

clf_rf = RandomForestClassifier(verbose=2, n_jobs=-1)
%time clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)

print('fraction:', fraction)
print(clf_rf.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')

# print(clf_rf.feature_importances_)
feature_importances(clf_rf)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
Wall time: 122 ms
fraction: 3000
0.6842105263157895 

[[79 17  1]
 [10 37  6]
 [12 14 14]] 



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


[(0.15124166349572132, 'dr_min'),
 (0.13911552474492936, 'ts_len'),
 (0.13306980601259982, 'ts_density'),
 (0.1330435733342815, 'dr_mean'),
 (0.09332205625897481, 'ts_diff'),
 (0.09217985895197833, 'dr_max'),
 (0.08131393968308762, 'dr_var'),
 (0.07115295678843994, 'ts_min'),
 (0.06517502923634497, 'ts_max'),
 (0.04038559149364236, 'dr_median')]

In [12]:
# ExtraTrees Classifier

clf_ext = ExtraTreesClassifier(bootstrap=False, criterion='gini', 
                               max_features=0.1, min_samples_leaf=3, 
                               min_samples_split=2, n_estimators=100,
                               n_jobs=-1)

%time clf_ext.fit(X_train, y_train)
y_pred = clf_ext.predict(X_test)

print('fraction:', fraction)
print(clf_ext.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')

# print(feature_importances(clf.ext))
# print(clf_rf.feature_importances_)
feature_importances(clf_ext)

Wall time: 261 ms
fraction: 3000
0.5684210526315789 

[[94  3  0]
 [42  9  2]
 [30  5  5]] 



[(0.2030854765041862, 'ts_diff'),
 (0.15215514555888, 'dr_mean'),
 (0.138368921783162, 'ts_len'),
 (0.11872177321777773, 'ts_density'),
 (0.10700378590321052, 'dr_median'),
 (0.10100009313311781, 'dr_max'),
 (0.08432071734681061, 'dr_min'),
 (0.03978267916961247, 'dr_var'),
 (0.02891594848976531, 'ts_max'),
 (0.026645458893477483, 'ts_min')]

In [13]:
# ExtraTrees Classifier

clf_ext = ExtraTreesClassifier(bootstrap=False, class_weight=None, 
                               criterion='gini', max_depth=None, 
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, min_samples_leaf=1, 
                               min_samples_split=2, 
                               min_weight_fraction_leaf=0.0, n_estimators=10, 
#                                n_jobs=None, 
                               n_jobs=-1, 
                               oob_score=False, 
                               random_state=None, verbose=0, warm_start=False)

%time clf_ext.fit(X_train, y_train)
y_pred = clf_ext.predict(X_test)

print('fraction:', fraction, 'score:', clf_ext.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')
feature_importances(clf_ext)

Wall time: 120 ms
fraction: 3000 score: 0.7 

[[84 11  2]
 [13 34  6]
 [11 14 15]] 



[(0.1623521342553966, 'dr_mean'),
 (0.14013652874408591, 'ts_len'),
 (0.13015234254085245, 'dr_min'),
 (0.10255225281182537, 'ts_density'),
 (0.09739415989118606, 'ts_diff'),
 (0.09059227814903806, 'dr_max'),
 (0.07449106539411157, 'ts_max'),
 (0.07281989409227019, 'dr_var'),
 (0.0681674253347797, 'ts_min'),
 (0.061341918786454154, 'dr_median')]

In [14]:
# # ExtraTrees Classifier

clf_ext = ExtraTreesClassifier(bootstrap=False, class_weight=None, 
                               criterion='gini', max_depth=None, 
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, min_samples_leaf=1, 
                               min_samples_split=2, 
                               min_weight_fraction_leaf=0.0, n_estimators=10, 
                               n_jobs=None, oob_score=False, 
                               random_state=None, verbose=0, warm_start=False)

%time clf_ext.fit(X_train, y_train)
y_pred = clf_ext.predict(X_test)

print('fraction:', fraction, 'score:', clf_ext.score(X_test, y_test), '\n')
print(confusion_matrix(y_test, y_pred), '\n')
feature_importances(clf_ext)

Wall time: 25.9 ms
fraction: 3000 score: 0.6789473684210526 

[[83 13  1]
 [15 30  8]
 [ 9 15 16]] 



[(0.15819949547953, 'dr_mean'),
 (0.13032891710298433, 'dr_min'),
 (0.12350387437887289, 'ts_len'),
 (0.12283959005184404, 'ts_density'),
 (0.10195948647849665, 'dr_max'),
 (0.09958353620460611, 'ts_diff'),
 (0.07065322766565044, 'dr_var'),
 (0.06482618950415252, 'ts_max'),
 (0.06423852908167482, 'dr_median'),
 (0.06386715405218825, 'ts_min')]

In [15]:
# # XGBClassifier
# param = {
#     'tree_method': 'gpu_hist', 
#     'gpu_id': 0
# }
# clf_xgb = xgb.XGBClassifier(**param)

# %time clf_xgb.fit(X_train, y_train)
# y_pred = clf_xgb.predict(X_test)

# print('fraction:', fraction, 'score:', clf_xgb.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# feature_importances(clf_xgb)

In [16]:
# # XGBClassifier
# # param = {
# #     'tree_method': 'gpu_hist', 
# #     'gpu_id': 0
# # }
# clf_xgb = xgb.XGBClassifier()

# %time clf_xgb.fit(X_train, y_train)
# y_pred = clf_xgb.predict(X_test)

# print('fraction:', fraction, 'score:', clf_xgb.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# feature_importances(clf_xgb)

In [17]:
# # XGBClassifier

# # clf_xgb = XGBClassifier()
# clf_xgb = xgb.XGBClassifier(base_score=0.5, booster='gbtree', 
#                             colsample_bylevel=1, colsample_bynode=1, 
#                             colsample_bytree=1, gamma=0, learning_rate=0.1, 
#                             max_delta_step=0, max_depth=3, min_child_weight=1,
#                             missing=None, n_estimators=100, n_jobs=1, 
# #                             nthread=None, 
#                             nthread=None, objective='multi:softprob', 
#                             random_state=0, reg_alpha=0, reg_lambda=1, 
#                             scale_pos_weight=1, seed=None, silent=None, 
#                             subsample=1, verbosity=1)

# %time clf_xgb.fit(X_train, y_train)
# y_pred = clf_xgb.predict(X_test)

# print('fraction:', fraction, 'score:', clf_xgb.score(X_test, y_test), '\n')
# print(confusion_matrix(y_test, y_pred), '\n')
# feature_importances(clf_xgb)

In [18]:
# def load_DMatrix(Xt: 'np.array', yt: 'np.array', 
#                  frac: int) -> Tuple['np.array']:
#     step, i = Xt.size // frac, 0
    
#     while i <= frac:
#         st, ed = i * step, (i + 1) * step
        
# #         yield Xt[st: ed], yt[st: ed]
#         yield xgb.DMatrix(Xt[st: ed], label=yt[st: ed])
        
#         i += 1

In [19]:
# for (x, y) in load_DMatrix(arr_X, arr_y, fraction):
#     print(x.size)
#     print(y.size)
# #     rret
# # 0
# # 1717435

In [20]:
# # Specify sufficient boosting iterations to reach a minimum
# num_round = 3

# # Leave most parameters as default
# param = {
# #     'subsample': 1 / fraction,
#     'objective': 'multi:softmax',  # Specify multiclass classification
#     'num_class': 3,  # Number of possible output classes
#     'tree_method': 'gpu_hist'  # Use GPU accelerated algorithm
# }

# # Convert input data from numpy to XGBoost format
# dtrain = xgb.DMatrix(X_train, label=y_train, nthread=-1)
# dtest = xgb.DMatrix(X_test, label=y_test, nthread=-1)
# # evallist = [(dtrain, 'train'), (dtest, 'test')]
# evallist = [(dtest, 'test')]
# gpu_res = {}  # Store accuracy result

# # Train model
# tmp = time.time()
# bst = xgb.train(param, 
#                 dtrain, 
# #                 load_DMatrix(arr_X, arr_y, fraction),
#                 num_boost_round=num_round, 
#                 evals=evallist, evals_result=gpu_res)
# print(f'GPU Training Time: {time.time() - tmp} seconds')

# # path = r''
# # booster.save_binary(dmatrix_test_filename)

# # save model
# bst.save_model(os.path.join(model_folder, 'xgb_70.model'))
# # dump model
# bst.dump_model(os.path.join(model_folder, 'xgb_70.raw.txt'))

# # dump model with feature map
# # booster.dump_model('xgb_70.nice.txt', '../data/featmap.txt')
# # save dmatrix into binary buffer（数据集保存）
# # dtest.save_binary('dtest.buffer')

# # Repeat for CPU algorithm
# # tmp = time.time()
# # param['tree_method'] = 'hist'
# # cpu_res = {}
# # xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_res)
# # print("CPU Training Time: %s seconds" % (str(time.time() - tmp)))

In [21]:
# y_pred = bst.predict(dtest, ntree_limit=num_round)
# conf = confusion_matrix(y_test, y_pred)
# score = np.sum(conf.diagonal()) / np.sum(conf)
# print(conf, '\n')
# print('fraction:', fraction, 'score:', score, '\n')
# # print(feature_importances(booster))

In [22]:
# md_pa = os.path.join(model_folder, r'model_no-settings.p')
# with gzip.GzipFile(md_pa, 'wb') as file:
#     pickle.dump(clf_xgb, file)

In [23]:
# best_type = clf.fitted_pipeline_.steps[1]
# print(best_type, '\n')

# %time best_type.fit(X_train, y_train)
# # y_pred = best_type.predict(X_test)

# print(best_type.score(X_test, y_test), '\n')
# # print(confusion_matrix(y_test, y_pred), '\n')
# print(best_type.feature_importances_)

# md_pa = os.path.join(model_folder, r'model_clf2.dat')
# with open(md_pa, 'wb') as file:
#     pickle.dump(best_type, file)